In [ ]:
import rdkit
from rdkit.Chem import AllChem
import pandas as pd
import os
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit import Chem
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import Draw
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import DataStructs

In [ ]:
import json
with open('../../sup_info/crossdock2020_duplicated_uniprotId_map_smiles_in_trainset_inchi_map.json','r') as f:
    crossdock2020_duplicated_uniprotId_map_smiles_in_trainset = json.load(f)


In [ ]:


def get_trainset_scaffold(x,ref_map):
    uniprot_id  = x['UniprotID'].split('_')[0]
    if uniprot_id not in ref_map:
        return []
    else:
        all_find_inchi = [i[1:-1] for i in x.Finded_Scaffolds_Inchi[1:-1].split(', ')]

        all_ref_inchi = ref_map[uniprot_id+'_scaffold'].keys()

        list_dup_inchi = list(set(all_find_inchi).intersection(set(all_ref_inchi)))
        # list_dup_scaffolds = [i for i in list_dup_scaffolds if i != '']
        list_dup_scaffolds = set([ref_map[uniprot_id+'_scaffold'][i] for i in list_dup_inchi])

        return list_dup_scaffolds
def get_trainset_smiles(x,ref_map):
    uniprot_id  = x['UniprotID'].split('_')[0]
    if uniprot_id not in ref_map:
        return []
    else:
        all_find_inchi = [i[1:-1] for i in x.Finded_Inchi[1:-1].split(', ')]

        all_ref_inchi = ref_map[uniprot_id].keys()
        list_dup_inchi = list(set(all_find_inchi).intersection(set(all_ref_inchi)))
        list_dup_smiles = set([ref_map[uniprot_id][i] for i in list_dup_inchi])
    
        return list_dup_smiles

In [ ]:

def getScaffoldAndSmiles(all_result_path):
    all_results = os.listdir(all_result_path)
    all_results_pd = []

    for temp_path in all_results:
        
        model_name = os.path.splitext(temp_path)[0]

        temp_path = os.path.join(all_result_path, temp_path)
        temp_pd = pd.read_csv(temp_path)
        temp_pd['ModelName'] = [model_name]*len(temp_pd)
        temp_pd['FindedScaffoldIsInTrainset'] = temp_pd.apply(lambda x :len(get_trainset_scaffold(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)), axis=1)
        temp_pd['FindedSmilesIsInTrainset'] = temp_pd.apply(lambda x :len(get_trainset_smiles(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)), axis=1)
        
        
        all_results_pd.append(temp_pd)
    merged_result_pd = pd.concat(all_results_pd,axis = 0)
    
        

    merged_result_pd['Finded_Scaffolds_Num'] = merged_result_pd['Finded_Scaffolds_Num'] - merged_result_pd['FindedScaffoldIsInTrainset']
    merged_result_pd['Finded_Smiles_Num'] = merged_result_pd['Finded_Smiles_Num'] - merged_result_pd['FindedSmilesIsInTrainset']
    
    scaffold_molecule_pd = merged_result_pd.groupby('ModelName').agg(list)[['Finded_Smiles_Num','Finded_Scaffolds_Num']].reset_index()#.apply(lambda x: sum(x)/120,axis = 1).plot(kind = 'bar',figsize=(10,6))
    

    # ## 统计找到骨架和找到分子的靶点数目
    scaffold_molecule_pd['Number_of_Finded_Scaffolds'] = scaffold_molecule_pd['Finded_Scaffolds_Num'].apply(lambda x: np.sum(np.array(x, dtype=bool)))
    scaffold_molecule_pd['Number_of_Finded_Smiles'] = scaffold_molecule_pd['Finded_Smiles_Num'].apply(lambda x: np.sum(np.array(x, dtype=bool)))
    return scaffold_molecule_pd
    
    


In [ ]:
all_result_path1 = '/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/Hit_Info_Results'
scaffold_molecule_pd1   =  getScaffoldAndSmiles(all_result_path1)
all_result_path2 = '/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/Hit_Info_Results'
scaffold_molecule_pd2   =  getScaffoldAndSmiles(all_result_path2)
all_result_path3 = '/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/Hit_Info_Results'
scaffold_molecule_pd3   =  getScaffoldAndSmiles(all_result_path3)


In [ ]:
scaffold_molecule_pd_repeats = pd.concat([scaffold_molecule_pd1,scaffold_molecule_pd2,scaffold_molecule_pd3],axis = 0)
# scaffold_molecule_pd_repeats[['Number_of_Finded_Scaffolds', 'Number_of_Finded_Smiles']] = scaffold_molecule_pd_repeats[['Number_of_Finded_Scaffolds', 'Number_of_Finded_Smiles']] /6
scaffold_molecule_pd_repeats = scaffold_molecule_pd_repeats[['ModelName','Number_of_Finded_Scaffolds', 'Number_of_Finded_Smiles']].groupby('ModelName').agg(['mean','std']).reset_index()
scaffold_molecule_pd_repeats.columns = ['ModelName','Number_of_Finded_Scaffolds_mean','Number_of_Finded_Scaffolds_std','Number_of_Finded_Smiles_mean','Number_of_Finded_Smiles_std']
scaffold_molecule_pd_repeats.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)

In [ ]:
scaffold_molecule_pd_repeats.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False).to_csv('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_all_protein.csv',index = False)

In [ ]:
import os
import pandas as pd

scaffold_molecule_pd_repeats.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False).to_csv('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_all_protein.csv',index = False)

In [ ]:

scaffold_molecule_pd = scaffold_molecule_pd_repeats.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))
color_list = sns.color_palette("Set2", n_colors=7)
# Create a color mapping for each model name
model_names = sorted(scaffold_molecule_pd['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}

model_colors = [color_map[name] for name in scaffold_molecule_pd['ModelName']]
# color_map 
bars1 = plt.bar(scaffold_molecule_pd['ModelName'],scaffold_molecule_pd['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6,color=model_colors )
bars2 = plt.bar(scaffold_molecule_pd['ModelName'],scaffold_molecule_pd['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///",color = model_colors, )
# scaffold_molecule_pd[['Number_of_Finded_Scaffolds']].plot(kind = 'bar',xlabel = 'ModelName',ylabel = 'Number of Targets',label = 'Number of Finded Scaffolds')
# scaffold_molecule_pd[['Number_of_Finded_Smiles']].plot(kind = 'bar',label = 'Number of Finded Smiles')
_ = plt.xticks(range(len(scaffold_molecule_pd['ModelName'])),scaffold_molecule_pd['ModelName'],rotation =45,ha='right')


for bar in bars1:
    
    
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{round(float(height),3)}', ha='center', va='bottom')
for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{round(float(height),3)}', ha='center', va='bottom')
    
# 绘制误差条
plt.errorbar(scaffold_molecule_pd['ModelName'], scaffold_molecule_pd['Number_of_Finded_Scaffolds_mean'], 
             yerr=scaffold_molecule_pd['Number_of_Finded_Scaffolds_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

plt.errorbar(scaffold_molecule_pd['ModelName'], scaffold_molecule_pd['Number_of_Finded_Smiles_mean'], 
             yerr=scaffold_molecule_pd['Number_of_Finded_Smiles_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

# Add padding to y-axis
max_val = scaffold_molecule_pd['Number_of_Finded_Scaffolds_mean'].max()
plt.ylim(0, )  # Add 20% padding to the top
    
# plt.ylabel('Number of Targets')
plt.title('All Proteins')
# plt.legend()
plt.ylabel('Number of Serises')
plt.legend(loc='upper right')
plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_all_protein.svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()

In [ ]:



scaffold_molecule_pd = scaffold_molecule_pd_repeats.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))
color_list = sns.color_palette("Set2", n_colors=7)
# Create a color mapping for each model name
model_names = sorted(scaffold_molecule_pd['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}

model_colors = [color_map[name] for name in scaffold_molecule_pd['ModelName']]
# color_map 
bars1 = plt.bar(scaffold_molecule_pd['ModelName'],scaffold_molecule_pd['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6,color=model_colors )
bars2 = plt.bar(scaffold_molecule_pd['ModelName'],scaffold_molecule_pd['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///",color = model_colors, )
# scaffold_molecule_pd[['Number_of_Finded_Scaffolds']].plot(kind = 'bar',xlabel = 'ModelName',ylabel = 'Number of Targets',label = 'Number of Finded Scaffolds')
# scaffold_molecule_pd[['Number_of_Finded_Smiles']].plot(kind = 'bar',label = 'Number of Finded Smiles')
_ = plt.xticks(range(len(scaffold_molecule_pd['ModelName'])),scaffold_molecule_pd['ModelName'],rotation =45,ha='right')

plt.errorbar(scaffold_molecule_pd['ModelName'], scaffold_molecule_pd['Number_of_Finded_Scaffolds_mean'], 
             yerr=scaffold_molecule_pd['Number_of_Finded_Scaffolds_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

plt.errorbar(scaffold_molecule_pd['ModelName'], scaffold_molecule_pd['Number_of_Finded_Smiles_mean'], 
             yerr=scaffold_molecule_pd['Number_of_Finded_Smiles_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

    
    

plt.title('All Proteins')
# plt.legend()
plt.ylim(0, )  # Add 20% padding to the top
plt.ylabel('Number of Serises')

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_all_protein_no_text.svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()

### 见过的蛋白结果

In [ ]:
# all_result_path = '/home/datahouse1/caoduanhua/MolGens/SelfConstructedBenchmark/AnalysisResults/hit_to_lead_scaffold_recovery/withFixedStero/results/Round1'
import json
with open('../../sup_info/UniprotIDs_duplicated_with_crossdock2020.json','r') as f:
    UniprotId_in_crossdock = json.load(f)
def getScaffoldAndSmilesSeen(all_result_path,UniprotId_in_crossdock,seen=True):
    all_results = os.listdir(all_result_path)
    all_results_pd = []

    for temp_path in all_results:
        
        model_name = os.path.splitext(temp_path)[0]
        # if model_name == 'PGMG':

        #     temp_path = os.path.join(all_result_path.replace('withFixedStero','withoutFixedStero'), temp_path)
        # else:
        temp_path = os.path.join(all_result_path, temp_path)
        temp_pd = pd.read_csv(temp_path)
        temp_pd['ModelName'] = [model_name]*len(temp_pd)
        all_results_pd.append(temp_pd)
    merged_result_pd = pd.concat(all_results_pd,axis = 0)
    merged_result_pd['Dupliceted_UniprotID'] = merged_result_pd['UniprotID'].apply(lambda x: x.split('_')[0] in UniprotId_in_crossdock)
    if seen:
        merged_result_pd = merged_result_pd[merged_result_pd['Dupliceted_UniprotID']]
        print('Number of targets: ',len(merged_result_pd['UniprotID'].unique()))
    else:
        merged_result_pd = merged_result_pd[~merged_result_pd['Dupliceted_UniprotID']]
        print('Number of targets: ',len(merged_result_pd['UniprotID'].unique()))
    
    
    scaffold_molecule_pd = merged_result_pd.groupby('ModelName').agg(list)[['Finded_Smiles_Num','Finded_Scaffolds_Num']].reset_index()#.apply(lambda x: sum(x)/120,axis = 1).plot(kind = 'bar',figsize=(10,6))

    #  统计找到骨架和找到分子的靶点数目
    scaffold_molecule_pd['Number_of_Finded_Scaffolds'] = scaffold_molecule_pd['Finded_Scaffolds_Num'].apply(lambda x: np.sum(np.array(x, dtype=bool)))
    scaffold_molecule_pd['Number_of_Finded_Smiles'] = scaffold_molecule_pd['Finded_Smiles_Num'].apply(lambda x: np.sum(np.array(x, dtype=bool)))
    return scaffold_molecule_pd

In [ ]:
scaffold_molecule_pd1_seen = getScaffoldAndSmilesSeen(all_result_path1,UniprotId_in_crossdock,seen=True)
scaffold_molecule_pd2_seen = getScaffoldAndSmilesSeen(all_result_path2,UniprotId_in_crossdock,seen=True)
scaffold_molecule_pd3_seen = getScaffoldAndSmilesSeen(all_result_path3,UniprotId_in_crossdock,seen=True)
scaffold_molecule_pd_repeats_seen = pd.concat([scaffold_molecule_pd1_seen,scaffold_molecule_pd2_seen,scaffold_molecule_pd3_seen],axis = 0)
scaffold_molecule_pd_repeats_seen = scaffold_molecule_pd_repeats_seen[['ModelName','Number_of_Finded_Scaffolds', 'Number_of_Finded_Smiles']].groupby('ModelName').agg(['mean','std']).reset_index()
scaffold_molecule_pd_repeats_seen.columns = ['ModelName','Number_of_Finded_Scaffolds_mean','Number_of_Finded_Scaffolds_std','Number_of_Finded_Smiles_mean','Number_of_Finded_Smiles_std']
scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)

In [ ]:
scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False).to_csv('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_seen_protein.csv',index = False)

In [ ]:
import pandas as pd
scaffold_molecule_pd_repeats_seen = pd.read_csv("/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_seen_protein.csv")

In [ ]:

scaffold_molecule_pd_seen = scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))

color_list = sns.color_palette("Set2", n_colors=7)
# Create a color mapping for each model name
model_names = sorted(scaffold_molecule_pd_seen['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}

model_colors = [color_map[name] for name in scaffold_molecule_pd_seen['ModelName']]
# color_map 
bars1 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6,color=model_colors )
bars2 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///",color = model_colors, )

_ = plt.xticks(range(len(scaffold_molecule_pd_seen['ModelName'])),scaffold_molecule_pd_seen['ModelName'],rotation =45,ha='right',fontsize=16)
# 在柱子上添加具体数值 并且显示方差数据

for bar in bars1:
    
    
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.8, f'{round(float(height),3)}', ha='center', va='bottom')
for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.8, f'{round(float(height),3)}', ha='center', va='bottom')
    


for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'],
    scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_std']
)):
    if not np.isnan(std):
        bar = bars1[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean

        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)

        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'],
    scaffold_molecule_pd_seen['Number_of_Finded_Smiles_std']
)):
    if not np.isnan(std):
        bar = bars2[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean
        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)
        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

# ==============================
    
    
plt.ylim(0, )  # Add 20% padding to the top
plt.ylabel('Number of Serises')
plt.title('Proteins in CrossDock2020(Seen)')
plt.legend()

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/11-hit2lead_mol_scaffold_recovery_seen_protein_fzh.svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()

In [ ]:

scaffold_molecule_pd_seen = scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))

color_list = sns.color_palette("Set2", n_colors=7)
# Create a color mapping for each model name
model_names = sorted(scaffold_molecule_pd_seen['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}

model_colors = [color_map[name] for name in scaffold_molecule_pd_seen['ModelName']]
# color_map 
bars1 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6,color=model_colors )
bars2 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///",color = model_colors, )

_ = plt.xticks(range(len(scaffold_molecule_pd_seen['ModelName'])),scaffold_molecule_pd_seen['ModelName'],rotation =45,ha='right')

for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'],
    scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_std']
)):
    if not np.isnan(std):
        bar = bars1[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean

        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)

        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'],
    scaffold_molecule_pd_seen['Number_of_Finded_Smiles_std']
)):
    if not np.isnan(std):
        bar = bars2[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean
        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)
        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

# ==============================
    
    
plt.ylim(0, )  # Add 20% padding to the top
plt.ylabel('Number of Serises')
plt.title('Proteins in CrossDock2020(Seen)')
# plt.legend()

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/11-hit2lead_mol_scaffold_recovery_seen_protein_no_text_fzh.svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()

### 没见过的蛋白结果

In [ ]:

scaffold_molecule_pd1_unseen = getScaffoldAndSmilesSeen(all_result_path1,UniprotId_in_crossdock,seen=False)
scaffold_molecule_pd2_unseen = getScaffoldAndSmilesSeen(all_result_path2,UniprotId_in_crossdock,seen=False)
scaffold_molecule_pd3_unseen = getScaffoldAndSmilesSeen(all_result_path3,UniprotId_in_crossdock,seen=False)
scaffold_molecule_pd_repeats_unseen = pd.concat([scaffold_molecule_pd1_unseen,scaffold_molecule_pd2_unseen,scaffold_molecule_pd3_unseen],axis = 0)
scaffold_molecule_pd_repeats_unseen = scaffold_molecule_pd_repeats_unseen[['ModelName','Number_of_Finded_Scaffolds', 'Number_of_Finded_Smiles']].groupby('ModelName').agg(['mean','std']).reset_index()
scaffold_molecule_pd_repeats_unseen.columns = ['ModelName','Number_of_Finded_Scaffolds_mean','Number_of_Finded_Scaffolds_std','Number_of_Finded_Smiles_mean','Number_of_Finded_Smiles_std']
scaffold_molecule_pd_repeats_unseen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)

In [ ]:
scaffold_molecule_pd_repeats_unseen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False).to_csv('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_unseen_protein.csv',index = False)

In [ ]:
import pandas as pd

scaffold_molecule_pd_repeats_unseen = pd.read_csv("/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/hit2lead_mol_scaffold_recovery_unseen_protein.csv")

In [ ]:

scaffold_molecule_pd_unseen = scaffold_molecule_pd_repeats_unseen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))

color_list = sns.color_palette("Set2", n_colors=7)
# Create a color mapping for each model name
model_names = sorted(scaffold_molecule_pd_unseen['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}

model_colors = [color_map[name] for name in scaffold_molecule_pd_unseen['ModelName']]
# color_map 
bars1 = plt.bar(scaffold_molecule_pd_unseen['ModelName'],scaffold_molecule_pd_unseen['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6,color=model_colors )
bars2 = plt.bar(scaffold_molecule_pd_unseen['ModelName'],scaffold_molecule_pd_unseen['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///",color = model_colors, )

_ = plt.xticks(range(len(scaffold_molecule_pd_unseen['ModelName'])),scaffold_molecule_pd_unseen['ModelName'],rotation =45,ha='right')


for bar in bars1:
    
    
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{round(float(height),3)}', ha='center', va='bottom')
for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{round(float(height),3)}', ha='center', va='bottom')

for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_unseen['Number_of_Finded_Scaffolds_mean'],
    scaffold_molecule_pd_unseen['Number_of_Finded_Scaffolds_std']
)):
    if not np.isnan(std):
        bar = bars1[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean
        # 上半段误差线
        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)
        # 顶端短横线
        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_unseen['Number_of_Finded_Smiles_mean'],
    scaffold_molecule_pd_unseen['Number_of_Finded_Smiles_std']
)):
    if not np.isnan(std):
        bar = bars2[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean
        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)
        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

# ==============================
    
plt.ylim(0, )  # Add 20% padding to the top
plt.ylabel('Number of Serises')
plt.title('Proteins in CrossDock2020(UnSeen)')
plt.legend()

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/11-hit2lead_mol_scaffold_recovery_unseen_protein_fzh.svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()

In [ ]:

scaffold_molecule_pd_unseen = scaffold_molecule_pd_repeats_unseen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))

color_list = sns.color_palette("Set2", n_colors=7)
# Create a color mapping for each model name
model_names = sorted(scaffold_molecule_pd_unseen['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}

model_colors = [color_map[name] for name in scaffold_molecule_pd_unseen['ModelName']]
# color_map 
bars1 = plt.bar(scaffold_molecule_pd_unseen['ModelName'],scaffold_molecule_pd_unseen['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6,color=model_colors )
bars2 = plt.bar(scaffold_molecule_pd_unseen['ModelName'],scaffold_molecule_pd_unseen['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///",color = model_colors, )

_ = plt.xticks(range(len(scaffold_molecule_pd_unseen['ModelName'])),scaffold_molecule_pd_unseen['ModelName'],rotation =45,ha='right')

for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_unseen['Number_of_Finded_Scaffolds_mean'],
    scaffold_molecule_pd_unseen['Number_of_Finded_Scaffolds_std']
)):
    if not np.isnan(std):
        bar = bars1[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean
        # 上半段误差线
        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)
        # 顶端短横线
        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

for i, (mean, std) in enumerate(zip(
    scaffold_molecule_pd_unseen['Number_of_Finded_Smiles_mean'],
    scaffold_molecule_pd_unseen['Number_of_Finded_Smiles_std']
)):
    if not np.isnan(std):
        bar = bars2[i]
        x = bar.get_x() + bar.get_width() / 2
        y = mean
        plt.plot([x, x], [y, y + std], color='black', linewidth=1.5)
        plt.plot([x - 0.05, x + 0.05], [y + std, y + std], color='black', linewidth=1.5)

# ==============================

    
plt.ylim(0, )  # Add 20% padding to the top
plt.ylabel('Number of Serises')
plt.title('Proteins in CrossDock2020(UnSeen)')
# plt.legend()

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_H2L/Round1/Hit_to_Lead_Results/11-hit2lead_mol_scaffold_recovery_unseen_protein_no_text_fzh.svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()